# Cleaning and Preprocessing the Full Data Frame

The data we use for our analysis comes from the [European Parliament Proceedings Parallel Corpus (1996-2011)](https://www.statmt.org/europarl/) and contains all speeches given in the European Parliament between 1996 and 2011, as well as metadata such as a speaker's name, party affiliation, and the date a speech was given. The raw data consists of roughly 10,000 individual `.txt` files that vary quite widely in terms of file structure and organization. For example, some `.txt` files represent only a single contribution by a given MEP, while others contain all contributions made in a whole day. Because the data was collected over many years, there are also some inconsistencies when it comes to what metadata is available -- for example, some `.txt` files contain no party affiliation, and for others the party affiliation is listed under speaker name rather than the appropriate column designating party affiliation. Therefore, this notebook contains extensive cleaning and preprocessing steps necessary in order to transform the raw data into an organized data frame.

## Loading required packages

In [1]:
#load required packages
import pandas as pd
import numpy as np
import nltk
import os
import re
import string

## Properly Loading Data from .txt Files

The following cells of code first load all the `.txt` files into a single data frame, as well as clean portions of the data. In order to run the code below, we recommend downloading the re

In [2]:
#define functions for extracting data from .txt files

def date_extracter(x):
    return x.partition('ep-')[2][0:8]#.partition('.txt')[0]

def lang_extracter(x):
    return x.partition('LANGUAGE=\"')[2].partition('\"')[0]

def contribution_extracter(x):
    x = x.partition('>\n')[2].partition('\n\n<')[0]
    return x.replace('\n<P>\n', ' ')

def affiliation_extracter(row):
    x = row.partition('AFFILIATION="')[2].partition('"')[0]
    y = row.partition('NAME="')[2].partition('"')[0]
    if x == "":
        x = None
    if y == "":
        y = None
    return [x, y]

def clean_speaker_name(row):
    row = row.partition('"')[0].partition('(')[0].strip()
    return row

In [3]:
#define path to retrieve .txt files
path = f"{os.getcwd()}/raw_data"

In [4]:
#load data into corpus dictionary
corpus = {}
for i in os.listdir(path):
    corpus[i] = nltk.data.load(f'{path}/{i}', format='raw')
    
#transpose corpus
corpus = pd.DataFrame.from_dict([corpus]).T

#decode data from bytes to strings
success_counter = 0
failure_counter = 0
for index, i in enumerate(corpus[0]):
    try:
        corpus.iloc[index][0] = i.decode('UTF-8')
        success_counter += 1
    except UnicodeDecodeError:
        corpus.iloc[index][0] = i.decode('ISO-8859-1')
        failure_counter += 1
    
#create date column
corpus['txtfile'] = corpus.index
corpus['date'] = corpus['txtfile'].apply(date_extracter)
corpus['txt'] = corpus[0]
corpus = corpus.drop(columns = ['txtfile', 0])

In [5]:
corpus.head()

,date,txt
ep-11-03-08-009-06.txt,11-03-08,"<CHAPTER ID=""009-06"">\n6. Innovative financing..."
ep-08-02-20-004-02.txt,08-02-20,"<CHAPTER ID=""004-02"">\n2. (\n"
ep-07-01-17-011.txt,07-01-17,"<CHAPTER ID=""011"">\nDevelopment of the Communi..."
ep-10-09-21-015.txt,10-09-21,"<CHAPTER ID=""015"">\nInclusion of aviation acti..."
ep-06-04-04.txt,06-04-04,"<CHAPTER ID=""1"">\n<SPEAKER ID=""1"" LANGUAGE="""" ..."
...,...,...
ep-11-01-20-015.txt,11-01-20,"<CHAPTER ID=""015"">\nDecisions concerning certa..."
ep-11-05-10-013.txt,11-05-10,"<CHAPTER ID=""013"">\nCorrections to votes and v..."
ep-10-09-07-006-16.txt,10-09-07,"<CHAPTER ID=""006-16"">\n16. Role of women in an..."
ep-10-11-22-010.txt,10-11-22,"<CHAPTER ID=""010"">\nPetitions: see Minutes\n"


In [6]:
#check how many .txt files use UTF-8 decoding
success_counter

9672

In [7]:
#check how many .txt files use ISO-8859-1 decoding
failure_counter

1

In [8]:
#loop over every .txt file extracting data and parsing it into a list
list_of_dfs = []
counter = 0
for text in corpus['txt']:
    #split the .txt files into lists of chapters
    chap_split = text.split("<CHAP")

    #remove empty rows
    chap_split = [i for i in chap_split if i != '']

    #split each chapter into a list of contributions by chapter
    spea_split = []
    for i in chap_split:
        spea_split.append(i.split("SPEA"))

    #retrieve chapter ids from each chapter
    d = {}
    for chapter in spea_split:
        #careful: we assume that the chapter title is contained in the first element of the list 'chapter'
        d[chapter[0].partition('ID=')[2].partition('>\n')[0]] = []
        for contribution in chapter:
            d[chapter[0].partition('ID=')[2].partition('>\n')[0]].append(contribution)

    #retrieve chapter associated with each contribution
    entries = []
    chapter = []
    for i in d:
        entries.extend(d[i])
        for k in range(len(d[i])):
            chapter.append(i)

    #map contribution and chapter to a data frame
    results = pd.DataFrame({'contribution': entries,'chapter': chapter})

    #retrieve dictionary mapping chapter number to chapter titles
    chap_titles = {}
    for i in results.contribution:
        if 'TER ID=' in i:
            chap_titles[i.partition('ID=')[2].partition('>\n')[0]] = i.partition('>\n')[2].partition('\n<')[0]

    #convert chapter number to string in order to match with chapter title dictionary
    results['chapter'] = results['chapter'].astype(str)
    results['chapter_title'] = results['chapter'].map(chap_titles)

    #subset data frame to only include contributions, not chapters
    results = results[results.contribution.str.contains('KER ID')]

    #reset index
    results = results.reset_index(drop = True)

    #retrieve contribution ids and speaker names
    sp_id = {}
    dict_counter = 1
    for i in results.contribution:
        if i.partition('KER ID=')[2].partition(' ')[0] not in sp_id:
            sp_id[i.partition('KER ID=')[2].partition(' ')[0]] = i.partition('NAME=\"')[2].partition('\">')[0]
        else:
            sp_id[f"{i.partition('KER ID=')[2].partition(' ')[0]}_{dict_counter}"] = i.partition('NAME=\"')[2].partition('\">')[0]
            dict_counter += 1
            
    #create column for contribution ids
    results['contribution_id'] = sp_id.keys()

    #create column for speaker names
    results['speaker_name'] = results['contribution_id'].map(sp_id)

    #apply data extraction functions defined above

    #extract original language a speech was given in
    results['language'] = results['contribution'].apply(lang_extracter)

    #extract clean text
    results['contribution_text'] = results['contribution'].apply(contribution_extracter)

    #extract party affiliation
    results['party'] = results['contribution'].apply(affiliation_extracter)

    #extract date
    results['date'] = corpus['date'][counter]

    #extract contribution ids

    def id_extracter(x):
        return f"{corpus['date'][counter]}-{x}"

    results['contribution_id'] = results['contribution_id'].apply(id_extracter)

    #save data frame
    list_of_dfs.append(results)

    counter += 1

In [9]:
#concatenate individual pandas data frames for each .txt file into one big data frame
data = pd.concat(list_of_dfs, ignore_index=True)

In [ ]:
#create lowercase speaker name column
data["speaker_name_lower"] = data["speaker_name"].apply(str.lower)

#remove rows containing 'president' as speaker_name_lower
data = data[~data["speaker_name_lower"].str.contains('president')]

#further clean speaker names
data["speaker_name_lower"] = data["speaker_name_lower"].apply(clean_speaker_name)

#remove speaker_name_lower where the name ends in a comma
for index in data.index:
    if data.at[index, 'speaker_name_lower'] == ',':
        data.at[index, 'speaker_name_lower'] = data.at[index, 'speaker_name_lower'].replace(',', '')

## Assign MEPs to Party and Ideology

In [13]:
#load mep dataset
mep = pd.read_csv("/Users/konratpekkip/Downloads/MEP_dataframe.csv")

In [18]:
def party_separator_1(x):
    y = x[0]
    return y

def party_separator_2(x):
    y = x[1]
    return y

def replace_amp(row):
    if type(row[0]) is str:
        row[0] = row[0].replace('&amp;', '&')
    if type(row[1]) is str:
        row[1] = row[1].replace('&amp;', '&')
    return row

In [19]:
data['party'] = data['party'].apply(replace_amp)

In [20]:
data['party_test1'] = data['party'].apply(party_separator_1)
data['party_test2'] = data['party'].apply(party_separator_2)

In [21]:
data['party_test1'].value_counts()[:20]

PPE-DE                           8110
PPE                              6761
PSE                              5965
S&D                              5196
in writing                       4069
ALDE                             3776
NI                               3237
GUE/NGL                          2193
Verts/ALE                        2158
UEN                              1295
Member of the Commission         1254
ECR                              1229
in writing.                      1109
EFD                               947
(Applause)                        936
IND/DEM                           902
Commission                        850
rapporteur                        819
on behalf of the PPE-DE Group     487
on behalf of the PSE Group        456
Name: party_test1, dtype: int64

In [23]:
list_of_parties = ['PPE-DE', 
                   'PPE', 
                   'PSE', 
                   'S&D', 
                   'ALDE', 
                   'NI', 
                   'GUE/NGL', 
                   'Verts/ALE', 
                   'UEN', 
                   'ECR', 
                   'EFD', 
                   'IND/DEM',
                   'G/EFA',
                   'EUL/NGL',
                   'EPP',
                   'EPP-ED',
                   'NI/EFD']

In [24]:
#assign only clean party to party column
for index, row in enumerate(data['party_test1']):
    if type(row) is str:
        for party in list_of_parties:
            if party in row:
                data.iloc[index]['party'] = party
                
for index, row in enumerate(data['party_test2']):
    if type(row) is str:
        for party in list_of_parties:
            if party in row:
                data.iloc[index]['party'] = party

In [25]:
#count how many good rows vs bad rows we have

badcount = 0
goodcount = 0

for i in data['party']:
    if type(i) is list:
        badcount += 1
    if type(i) is str:
        goodcount += 1

In [26]:
goodcount

76915

In [27]:
badcount

103172

In [28]:
#assign labels to rows where party is clean vs not clean
data['party_confirmed'] = 0
for i in data.index:
    if type(data.at[i, 'party']) is list:
        data.at[i, 'party_confirmed'] = 0
    elif type(data.at[i, 'party']) is str:
        data.at[i, 'party_confirmed'] = 1

In [29]:
#extract rows from data frame that are ready to be used already and store them as good_data
bad_data = data[data['party_confirmed'] == 0]
good_data = data[data['party_confirmed'] == 1]

In [31]:
#reset index to remove index numbers from removed 'good data' rows
bad_data.reset_index(inplace = True, drop=True)

In [32]:
#remove rows without names
bad_data = bad_data[bad_data["speaker_name_lower"] != '']

In [34]:
#create column for lowercase versions of full mep names
mep['full_name_lower'] = mep['full_name'].apply(str.lower)

In [36]:
data['party_confirmed'] = 0
for i in data.index:
    if type(data.at[i, 'party']) is list:
        data.at[i, 'party_confirmed'] = 0
    elif type(data.at[i, 'party']) is str:
        data.at[i, 'party_confirmed'] = 1

In [37]:
for index in bad_data.index:
    if bad_data.at[index, 'speaker_name_lower'] == ',':
        bad_data.at[index, 'speaker_name_lower'] = bad_data.at[index, 'speaker_name_lower'].replace(',', '')

In [38]:
bad_data['match'] = 0
for index in bad_data.index:
    name = bad_data.at[index, 'speaker_name_lower']
    for mep_name in mep['full_name_lower']:
        if mep_name in name:
            bad_data.at[index, 'match'] = 1
        elif name in mep_name:
            bad_data.at[index, 'match'] = 1

In [40]:
matching_data = bad_data[bad_data['match'] == 1]

In [42]:
goodcount + len(matching_data)

142656

In [43]:
badcount - len(matching_data)

37431

In [44]:
#define a dictionary that maps a binary variable onto far-right vs non-far-right people
party_dictionary = {}

far_right_parties = ['UEN', 'EFD', 'NI', 'ECR', 'IND/DEM', 'NI/EFD']
not_far_right_parties = ['PPE-DE', 'PPE', 'PSE', 'S&D', 'ALDE', 'GUE/NGL', 'Verts/ALE', 'G/EFA', 'EUL/NGL', 'EPP', 'EPP-ED']

for i in list_of_parties:
    if i in far_right_parties:
        party_dictionary[i] = 1
    elif i in not_far_right_parties:
        party_dictionary[i] = 0

In [45]:
party_dictionary

{'PPE-DE': 0,
 'PPE': 0,
 'PSE': 0,
 'S&D': 0,
 'ALDE': 0,
 'NI': 1,
 'GUE/NGL': 0,
 'Verts/ALE': 0,
 'UEN': 1,
 'ECR': 1,
 'EFD': 1,
 'IND/DEM': 1,
 'G/EFA': 0,
 'EUL/NGL': 0,
 'EPP': 0,
 'EPP-ED': 0,
 'NI/EFD': 1}

In [46]:
#assign binary outcome to far-right and non-far-right parties   
good_data['far_right'] = [party_dictionary[x] for x in good_data['party']]

/var/folders/k7/w1b25s3n1md_4_385_qc_rqc0000gn/T/ipykernel_3753/1997053228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_data['far_right'] = [party_dictionary[x] for x in good_data['party']]


In [ ]:
#save good and bad data sets as csv files
#good_data.to_csv("good_data.csv")
#bad_data.to_csv("bad_data.csv")

In [47]:
matching_data.reset_index(drop = True, inplace = True)

In [52]:
#create subset of mep data frame for matching speaker_names in both matching_data and mep data frames
merge_mep = mep[['far_right', 'full_name_lower', 'epg_names', 'epg_code']]
merge_mep = merge_mep.drop_duplicates(['full_name_lower']).reset_index(drop=True)
merge_mep["full_name_lower"] = merge_mep["full_name_lower"].apply(clean_speaker_name)

In [98]:
#grab european party group from merge_mep dataframe to match speaker_names in matching_data with party assignment
#this takes a HOT MINUTE to run so do this sparingly

matching_data['party_from_mep'] = 'to be filled'

for index in matching_data.index:
    try:
        name = matching_data.at[index, 'speaker_name_lower']
    except:
        pass
    for index_2 in range(len(merge_mep)):#merge_mep.index:
        try:
            if merge_mep.at[index_2, 'full_name_lower'] in name:
                matching_data.at[index, 'party_from_mep'] = merge_mep.at[index_2, 'epg_names']
            elif name in merge_mep.at[index_2, 'full_name_lower']:
                matching_data.at[index, 'party_from_mep'] = merge_mep.at[index_2, 'epg_names']
        except:
            pass

In [113]:
#define which parties are far right

epg_far_right_parties = ['The European Conservatives and Reformists Group',
                         'Europe of Freedom and Democracy',
                         'Non-Inscrits',
                         "['Progressive European Democrats', 'European Democratic Alliance', 'Union for Europe', 'Union for a Europe of Nations']",
                         "['Technical Coordination of Democrats and Independents', 'Rainbow Group', 'European Radical Alliance']",
                         "['Independents for a European of Nations', 'Europe of Democracies and Diversities']",
                         "['Technical Coordination of Democrats and Independents', 'Non-attached', 'Technical Group of Independents']",
                         'Independence/Democracy',
                         'Non-Inscrits / Europe of Freedom and Democracy']



epg_not_far_right_parties = ["European People's Party Group",
                             'The Progressive Alliance of Socialists and Democrats',
                             'Alliance of Liberals and Democrats for Europe Party',
                             'The Greens/European Free Alliance',
                             'The Left in the European Parliament',
                             "['Socialist Group', 'Party of European Socialists']",
                             "['Communist Group', 'European United Left/Nordic Green Left', 'European United Left']",
                             '["European People\'s Party", "European People\'s Party-European Democrats"]',
                             "['Liberal and Democratic Group', 'Liberal Democratic and Reform Group', 'European Liberal Democratic and Reform Party']",
                             "['Green Group', 'Greens/European Free Alliance']",
                             'SOC',
                             "European People's Party Group and European Democrats"]

all_parties = epg_far_right_parties + epg_not_far_right_parties

In [137]:
#create dictionary of ep parties and ideological affiliation

party_dictionary_2 = {}

for i in all_parties:
    if i in epg_not_far_right_parties:
        party_dictionary_2[i] = 0
    if i in epg_far_right_parties:
        party_dictionary_2[i] = 1

In [139]:
#create binary far_right column based on dictionary defined above
matching_data['far_right'] = [party_dictionary_2[x] for x in matching_data['party_from_mep']]

/var/folders/k7/w1b25s3n1md_4_385_qc_rqc0000gn/T/ipykernel_3753/3356286057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_data['far_right'] = [party_dictionary_2[x] for x in matching_data['party_from_mep']]


In [153]:
#extract relevant columns for merging good_data and matching_data dfs
good_data_premerge = good_data[['contribution', 'chapter', 'chapter_title', 'contribution_id', 'speaker_name_lower', 'contribution_text', 'date', 'party', 'language', 'far_right']]
matching_data_premerge = matching_data[['contribution', 'chapter', 'chapter_title', 'contribution_id', 'speaker_name_lower', 'contribution_text', 'date', 'party_from_mep', 'language', 'far_right']]

#rename party_from_mep to be party in matching_data
matching_data_premerge.rename(columns = {'party_from_mep':'party'}, inplace = True)

In [171]:
#concatenate matching_data_premerge and good_data_premerge
final_data_list = [good_data_premerge, matching_data_premerge]
final_data = pd.concat(final_data_list, ignore_index=True)

In [177]:
#uncomment the next line to save final data to csv
#final_data.to_csv("final_data.csv")

In [178]:
test = final_data[final_data['far_right'] == 1]

In [179]:
test

,contribution,chapter,chapter_title,contribution_id,speaker_name_lower,contribution_text,date,party,language,far_right
27,"KER ID=""751"" NAME=""Jaroslav Paška"" AFFILIATION...","""015""",Inclusion of aviation activities in the scheme...,"10-09-21-""751""",jaroslav paška,(SK) I agree that reductions of emissions must...,10-09-21,EFD,,1
40,"KER ID=""24"" LANGUAGE="""" NAME=""Eoin Ryan,"" AFFI...","""1""",<,"06-04-04-""24""","eoin ryan,","<P>\n . Mr President, the broad economic pol...",06-04-04,UEN,,1
41,"KER ID=""25"" LANGUAGE="""" NAME=""Derek Roland Cla...","""1""",<,"06-04-04-""25""","derek roland clark,","<P>\n . Mr President, the report on employme...",06-04-04,IND/DEM,,1
42,"KER ID=""26"" LANGUAGE=""CS"" NAME=""Jana Bobošíkov...","""1""",<,"06-04-04-""26""",jana bobošíková,<P>\n The reports under discussion today fin...,06-04-04,NI,CS,1
47,"KER ID=""31"" LANGUAGE=""LV"" NAME=""Guntars Krasts...","""1""",<,"06-04-04-""31""",guntars krasts,"<P>\n Thank you, Mr President. First of all ...",06-04-04,UEN,LV,1
...,...,...,...,...,...,...,...,...,...,...
142634,"KER ID=219 NAME=""Scarbonchi"">\nArticles 85 and...",8,Competition in the air traffic sector,98-05-14-219,scarbonchi,Articles 85 and 86 of the Treaty of Rome provi...,98-05-14,['Technical Coordination of Democrats and Inde...,,1
142635,"KER ID=220 NAME=""Ribeiro"">\nMr President, bein...",8,Competition in the air traffic sector,98-05-14-220,ribeiro,"Mr President, being responsible for the opinio...",98-05-14,"['Progressive European Democrats', 'European D...",,1
142638,"KER ID=223 LANGUAGE=""DE"" NAME=""Lukas"">\nMr Pre...",8,Competition in the air traffic sector,98-05-14-223,lukas,"Mr President, the European Commission, when lo...",98-05-14,['Technical Coordination of Democrats and Inde...,DE,1
142648,"KER ID=241 LANGUAGE=""FR"" NAME=""Barthet-Mayer"">...",10,Marketing of seeds - Implementation of Regulat...,98-05-14-241,barthet-mayer,"Madam President, Commissioner, ladies and gent...",98-05-14,['Technical Coordination of Democrats and Inde...,FR,1


In [180]:
len(test) / len(final_data)

0.1750644908030507

In [182]:
final_data.iloc[4].contribution_text

"(PT) Once again, the majority in Parliament is trying to speed up the liberalisation and privatisation of rail passenger transport by seeking to broaden the scope of the proposed directive not only to international passenger transport, and by as early as 2010, but also to national passenger transport by 2017. Its aim is to deliver the most profitable lines into the hands of the large private economic operators by privatising services, specifically by means of public-private partnerships, thereby fostering, as has happened in other sectors, the formation of monopolies whose aim is profit. These monopolies often receive outlandish amounts of public funding, regardless of the interests and needs of each country and its people. In Portugal, experience has shown that, following the implementation of such a policy, public rail transport services have deteriorated, thereby hampering people's mobility. The policy has led to higher ticket prices, to the removal of hundreds of kilometres of the